In [1]:
import os
import csv
import math
import wandb
import numpy as np
from pathlib import Path 

import matplotlib.pyplot as plt

from keras.utils import np_utils
from keras.datasets import mnist
from keras.models import Sequential
from wandb.keras import WandbCallback
from keras.layers import Dense, Flatten, Dropout

Using TensorFlow backend.


In [2]:
#Getting the latest file from the folder
record_path = Path("RecordSummary")
training_data_list = list(filter(os.path.isfile,record_path.glob('*.csv')))
training_data_list.sort(key=lambda x: os.path.getmtime(x))
training_data = training_data_list[-1]

x_train_percent = 60
x_test_percent = 100-x_train_percent
x_train = []
x_test = []
y_train = []
y_test = []

with open(training_data) as csv_file:
    data = list(csv.reader(csv_file))
    lines = list(map(list, zip(*data)))

nr_x_train = math.ceil(len(lines[0])*x_train_percent/100)
lines = np.asarray(lines)

#X stands for input, Y stands for output

x_train = lines[1:, 0:nr_x_train]
y_train = lines[0, 0:nr_x_train]

x_test = lines[1:, nr_x_train:] 
y_test = lines[0, nr_x_train:]
x_train = x_train.transpose()
x_test = x_test.transpose()

x_train = [a.reshape(4,96) for a in x_train]
x_test = [a.reshape(4,96) for a in x_test]

is_up_y_train = y_train=='up'
is_up_y_test = y_test=='up'
labels = ["Down", "Up"]

In [3]:
channels = np.shape(x_train)[1]
samples = np.shape(x_train)[2]

print(channels)
print(samples)

4
96


In [4]:
print(np.shape(x_train))
print(np.shape(is_up_y_train))
print(np.shape(x_test))
print(np.shape(is_up_y_test))

(130, 4, 96)
(130,)
(86, 4, 96)
(86,)


In [5]:
print(type(x_train))
print(type(is_up_y_train))

<class 'list'>
<class 'numpy.ndarray'>


In [6]:
#converting <class 'list'> to <class 'numpy.ndarray'>
x_train = np.asarray(x_train)
x_test = np.asarray(x_test)

In [7]:
#Normalizing data:
x_train = x_train.astype("float")
x_test = x_test.astype("float")
merge = np.vstack((x_train, x_test))
minval = np.amin(merge)
maxval = np.amax(merge)
print(f'Min: {minval}, Max:{maxval}')
x_train = (x_train-minval)/(maxval-minval)
x_test = (x_test-minval)/(maxval-minval)

[['36.621' '31.25' '27.832' '25.391' '27.832' '34.18' '34.18' '25.879'
  '20.508' '20.508' '23.926' '27.344' '19.531' '17.578' '18.555' '20.508'
  '24.414' '25.879' '26.855' '25.391' '27.832' '31.25' '26.855' '21.973'
  '20.02' '27.832' '35.156' '32.227' '30.762' '34.18' '40.527' '38.086'
  '39.062' '39.062' '27.344' '32.227' '35.156' '25.879' '26.367' '32.715'
  '33.203' '28.32' '25.879' '27.344' '29.297' '27.832' '26.855' '24.414'
  '21.484' '23.926' '36.621' '39.551' '30.762' '24.414' '30.273' '36.621'
  '38.086' '39.551' '35.645' '29.297' '28.32' '34.18' '41.504' '34.18'
  '24.902' '27.832' '32.715' '32.715' '32.715' '35.156' '36.621' '41.992'
  '44.434' '39.062' '35.156' '31.738' '31.25' '38.086' '38.574' '37.598'
  '38.086' '36.133' '35.645' '36.133' '32.715' '26.855' '30.273' '35.645'
  '33.203' '27.832' '25.391' '26.855' '32.227' '37.598' '33.203' '26.367']
 ['31.738' '31.738' '29.297' '25.879' '25.879' '28.809' '31.738' '33.691'
  '33.691' '30.273' '30.762' '29.785' '26.367' '

In [ ]:
# create model
model=Sequential()
model.add(Flatten(input_shape=(channels,samples)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='mse', optimizer='adam',
                metrics=['accuracy'])

In [ ]:
# logging code
run = wandb.init(project="eeg-up-down")
config = run.config
config.epochs = 100
# Fit the model
model.fit(x_train, is_up_y_train, epochs=config.epochs, validation_data=(x_test, is_up_y_test),
                    callbacks=[WandbCallback(labels=labels)])

In [ ]:
print(model.predict(x_test[10:50,:,:]))
print(is_up_y_test[10:50])